In [ ]:
pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementNotInteractableException, ElementClickInterceptedException, StaleElementReferenceException

import time

In [10]:
def search_product_price(product_name):
    chrome_options = Options()
    chrome_options.add_argument("--auto-open-devtools-for-tabs")
    driver = webdriver.Chrome(options=chrome_options)
    wait = WebDriverWait(driver, 20)

    driver.get("https://www.pingodoce.pt/")
    time.sleep(3)

    try:
        # Fechar popup
        try:
            popup_close = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.icon-close-popup')))
            popup_close.click()
            print("Popup fechado")
        except:
            print("Popup não apareceu")

        # Aceitar cookies
        try:
            cookie_button = wait.until(EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler')))
            cookie_button.click()
            print("Cookies aceites")
        except:
            print("Barra de cookies não apareceu")

        # Esperar que o input esteja presente
        search_input = wait.until(EC.presence_of_element_located((By.ID, "header-search-bar")))
        print("Input de pesquisa localizado")

        # FORÇAR FOCUS e evento como se fosse inspecionado
        driver.execute_script("""
            var input = document.getElementById('header-search-bar');
            input.style.display = 'block';
            input.focus();
            input.dispatchEvent(new Event('focus', { bubbles: true }));
            input.dispatchEvent(new Event('click', { bubbles: true }));
        """)
        time.sleep(1)

        # Re-obter o elemento e escrever
        search_input = wait.until(EC.element_to_be_clickable((By.ID, "header-search-bar")))
        search_input.send_keys(product_name)
        search_input.send_keys(Keys.RETURN)
        print("Pesquisa enviada com sucesso")
        time.sleep(2)

        while True:
            try:
                ver_mais_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'load-more')]//button[contains(., 'Ver mais')]"))
                )
                driver.execute_script("arguments[0].scrollIntoView(true);", ver_mais_button)
                time.sleep(1)
                ver_mais_button.click()
                print("Clique em 'Ver mais'")
                time.sleep(2)
            except:
                print("Botão 'Ver mais' não encontrado ou já não existe")
                break

        # Extrair resultados
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.product-name-link')))
        time.sleep(2)

        # Capturar e filtrar visíveis com segurança
        all_products = driver.find_elements(By.CSS_SELECTOR, 'div.product-name-link')
        visible_products = []
        for p in all_products:
            try:
                if p.is_displayed():
                    visible_products.append(p)
            except StaleElementReferenceException:
                continue

        print(f"{len(visible_products)} produtos visíveis encontrados")


        results = []
        for product in all_products:
            try:
        # Extrair título
                title = product.find_element(By.CSS_SELECTOR, 'a').text.strip()

        # Extrair preço (tenta com sales -> value)
                try:
                    price = product.find_element(By.CSS_SELECTOR, 'span.value').text.strip()
                except:
                    price = "Preço não encontrado"

                results.append((title, price))

            except Exception as e:
                print("Erro ao extrair produto:", e)
                continue

        return results

    finally:
        driver.quit()

# Uso
product_name = input("Produto a procurar: ")
prices = search_product_price(product_name)

if prices:
    for title, price in prices:
        print(f"{title} - {price}")
else:
    print("Nenhum produto encontrado.")

Popup fechado
Cookies aceites
Input de pesquisa localizado
Pesquisa enviada com sucesso
Botão 'Ver mais' não encontrado ou já não existe
12 produtos visíveis encontrados
Leite UHT Meio Gordo - Preço não encontrado
Leite UHT Meio Gordo sem Lactose - Preço não encontrado
Leite UHT Meio Gordo - Preço não encontrado
Leite UHT Meio Gordo Pastagem - Preço não encontrado
Leite UHT Meio Gordo sem Lactose Mimosa - Preço não encontrado
Leite UHT Meio Gordo - Preço não encontrado
Leite UHT Meio Gordo - Preço não encontrado
Leite UHT Meio Gordo Pastagem Açores - Preço não encontrado
Leite UHT Meio Gordo Pack 3 - Preço não encontrado
Leite UHT Meio Gordo sem Lactose - Preço não encontrado
Leite UHT Meio Gordo Pack 4 - Preço não encontrado
Leite UHT Meio Gordo sem Lactose Pack 4 - Preço não encontrado
